In [15]:
import numpy as np
import time
from io import StringIO
import bqplot as bq
import ipywidgets as ipw
import ipyvolume as ipv
import IPython

In [2]:
def split_by_key(new, key, start_ind=0):
    "Split into before and after at first newline before `key`"
    # str.find returns -1 if not found
    ind = new[start_ind:].find(key)
    if ind != -1:
        # Find last \n before `key`
        reverse_str = new[ind-1::-1]
        reverse_ind = reverse_str.find('\n')
        # If \n isn't found, then all of `new` belongs
        # in the next group
        if reverse_ind == -1:
            newline_ind = 0
        else:
            newline_ind = ind - reverse_ind
            
        # Append text before `key` to `lines`
        before = new[:newline_ind]
        after = new[newline_ind:]
    else:
        before = new
        after = ''
        
    return before, after, ind


In [3]:
def parse_timestep(ts_string):
    lines = ts_string.split('\n')
    header = lines[:9]
    bounds_io = StringIO('\n'.join(lines[5:8]))
    data_io = StringIO('\n'.join(lines[9:]))
    bounds_arr = np.loadtxt(bounds_io) 
    try:
        data_arr = np.loadtxt(data_io)
    except ValueError:
        print("VE")
        IPython.embed()
    
    data_min, data_max = bounds_arr.T
    data_range = data_max - data_min
    
    atom_id, atom_type, xs, ys, zs, ix, iy, iz = data_arr.T
    sc = data_arr[:,[2,3,4]]
    im = data_arr[:,[5,6,7]]
    #pos = data_range * sc + data_min*im
    pos = sc
    
    return pos

In [4]:
def poll_file(path, func, key, poll_interval=1, ignore_first=True, *args, **kwargs):
    """
    Poll `path` every `poll_interval` seconds.
    Every time `key` is contained, call `func`,
    passing the full text read since the last time `key` was read
    as the argument. Then throw away what we've read before last `key`.
    
    args, kwargs will be passed to func.
    """
    
    lines = ''
    
    ignore = ignore_first
    
    while len(lines) == 0:
        try:
            with open(path) as fh:
                while True:
                    new = fh.read(256)
                    if len(new) > 0:
                        before, after, ind = split_by_key(new, key)
                        
                        # If `key` in `new`
                        if ind != -1:
                            if new.count(key) != 1:
                                print("count = {}".format(new.count(key)))
                                IPython.embed()
                            if ignore:
                                lines += new
                                ignore = False
                            else:
                                lines += before
                                func(lines, *args, **kwargs)
                                lines = after
                                    
                        else:
                            lines += new
                    else:
                        time.sleep(poll_interval)
                        
        except FileNotFoundError:
            print("No file yet.")
            time.sleep(1)


In [5]:
def g_of_r(atom_pos, dr, r_max):
    """
    atom_pos :: N x 3 array of atom positions
    dr :: bin thickness
    """
    
    n = atom_pos.shape[0]
    num_bins = int(np.floor(r_max / dr))
    r_vals = np.arange(0, r_max, dr)
    diff = np.zeros(n*(n-1))

    i, j = np.meshgrid(range(n), range(n))
    i = i.reshape(n**2)
    j = j.reshape(n**2)
    
    diff = np.linalg.norm(atom_pos[i,:] - atom_pos[j,:], axis=1)
    diff = diff[np.nonzero(diff)]
    
    g_vals, _ = np.histogram(diff, bins=num_bins, range=(0, r_max), density=True)
    
    return r_vals, g_vals

In [6]:
def create_bq_figure(x, y):
    xsc = bq.LinearScale()
    ysc = bq.LinearScale()
    xax = bq.Axis(scale=xsc, label='r')
    yax = bq.Axis(scale=ysc, orientation='vertical', label='g(r)')
    
    line = bq.Lines(
        x=x,
        y=y,
        scales={
            'x': xsc,
            'y': ysc
        }
    )
    
    fig = bq.Figure(
        axes = [xax, yax],
        marks = [line],
        #animation_duration=500
    )
    
    return line, fig

In [7]:
def create_ipv_scatter(xlim, ylim, zlim):
    fig = ipv.figure()
    scatter = ipv.scatter(*np.zeros([3,1]))
    
    fig.xlim = xlim
    fig.ylim = ylim
    fig.zlim = zlim
    
    fig.animation = 0
    
    return scatter, fig

In [8]:
def update_line(line, y):
    line.y = y

In [9]:
def update_ipv_scatter(scatter, atoms):
    scatter.x = atoms[:,0]
    scatter.y = atoms[:,1]
    scatter.z = atoms[:,2]

In [10]:
def update_atom_plots(atoms_str, line, scatter):
    atoms = parse_timestep(atoms_str)
    r, g = g_of_r(atoms, dr=.01, r_max=2)
    update_line(line, g)
    update_ipv_scatter(scatter, atoms)

In [12]:
r_max = 2
dr = .01
r = np.arange(0, r_max, dr)
g = np.zeros_like(r)

In [16]:
line, bq_fig = create_bq_figure(r, g)
scatter, ipv_fig = create_ipv_scatter(
    (0,1),
    (0,1),
    (0,1)
)

ipw.HBox([bq_fig, ipv_fig])

A Jupyter Widget

In [17]:
poll_file('lammps_melt/melt.lammpstrj', update_atom_plots, 'TIMESTEP', line=line, scatter=scatter)

No file yet.
No file yet.
No file yet.
VE
Python 3.5.3 |Continuum Analytics, Inc.| (default, Feb 22 2017, 21:13:27) 
Type 'copyright', 'credits' or 'license' for more information
IPython 6.2.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: len(lines)
Out[1]: 8019



KeyboardInterrupt: 

In [ ]:
# def poll_thermo(path, func, key):
#     lines = ''
#     
#     ignore = ignore_first
#     
#     while len(lines) == 0:
#         try:
#             with open(path) as fh:
#                 while True:
#                     new = fh.read(256)
#                     if len(new) > 0:
#                         before, after, ind = split_by_key(new, key)
#                         
#                         # If `key` in `new`
#                         if ind != -1:
#                             if ignore:
#                                 lines += new
#                                 ignore = False
#                             else:
#                                 lines += before
#                                 func(lines, *args, **kwargs)
#                                 lines = after
#                         else:
#                             lines += new
#                     else:
#                         time.sleep(poll_interval)
#                         
#         except FileNotFoundError:
#             print("No file yet.")
#             time.sleep(1)